# Advanced Data Processing with GPU Acceleration

This notebook demonstrates more advanced data processing techniques using GPU acceleration with RAPIDS cuDF.

In [ ]:
import cudf
import numpy as np
from time import time

# Create a large dataset
n_rows = 10_000_000
data = {
    'A': np.random.randn(n_rows),
    'B': np.random.randn(n_rows),
    'C': np.random.choice(['X', 'Y', 'Z'], n_rows),
    'D': np.random.randint(1, 100, n_rows)
}

# Create cuDF DataFrame
df_gpu = cudf.DataFrame(data)
print(f"Created DataFrame with {len(df_gpu):,} rows")

## Advanced GroupBy Operations

Let's explore some advanced groupby operations that demonstrate the power of GPU acceleration:

In [ ]:
# Complex groupby with multiple aggregations
start = time()
result = df_gpu.groupby('C').agg({
    'A': ['mean', 'std', 'min', 'max'],
    'B': ['sum', 'count'],
    'D': ['nunique', lambda x: x.quantile(0.95)]
}).reset_index()

print(f"Complex groupby operation completed in {time() - start:.2f} seconds")
print("\nResults:")
print(result)

## Window Functions

cuDF supports SQL-style window functions that can be incredibly powerful for time series analysis and other advanced operations:

In [ ]:
# Sort by A for window operations
df_gpu = df_gpu.sort_values('A')

# Calculate rolling mean with a window of 1000 rows
start = time()
df_gpu['A_rolling_mean'] = df_gpu['A'].rolling(window=1000).mean()

# Calculate cumulative sum within each group
df_gpu['B_cumsum'] = df_gpu.groupby('C')['B'].transform('cumsum')

# Calculate percent rank of D within each group
df_gpu['D_pctrank'] = df_gpu.groupby('C')['D'].transform(
    lambda x: (x.rank() - 1) / (len(x) - 1)
)

print(f"Window operations completed in {time() - start:.2f} seconds")
print("\nSample results:")
print(df_gpu.head())

## Advanced Joins and Merges

GPU-accelerated joins can be extremely fast, especially for large datasets:

In [ ]:
# Create a second DataFrame for joining
lookup_data = {
    'C': ['X', 'Y', 'Z'],
    'description': ['Group X', 'Group Y', 'Group Z'],
    'multiplier': [1.0, 1.5, 2.0]
}
lookup_df = cudf.DataFrame(lookup_data)

# Perform a left join
start = time()
enriched_df = df_gpu.merge(
    lookup_df,
    on='C',
    how='left'
)

# Calculate new values using joined data
enriched_df['weighted_value'] = enriched_df['A'] * enriched_df['multiplier']

print(f"Join and calculation completed in {time() - start:.2f} seconds")
print("\nSample results:")
print(enriched_df.head())

## Conclusion

In this notebook, we've explored several advanced data processing techniques using GPU acceleration with RAPIDS cuDF:

1. Complex groupby operations with multiple aggregations
2. Window functions including rolling statistics and group transforms
3. High-performance joins with large datasets

These operations can be orders of magnitude faster than traditional CPU-based processing, especially with large datasets.